In [3]:
import os
import pickle
import nd2
import numpy as np
import napari
from napari import Viewer
from magicgui import magicgui
from skimage.io import imsave
from skimage.measure import find_contours, approximate_polygon
from cardiomyocytes_helper_functions import create_mask_from_shapes,fill_gaps_between_cells 

In [4]:
def correct_shapes(viewer: Viewer):
    
    # get coordinates of vertices
    vertices_polygons = viewer.layers['Shapes'].data

    # correction only makes sense when there are more than one region
    if len(vertices_polygons) > 1:
    
        # create mask from vertices
        mask_shapes_overlap, mask_shapes = create_mask_from_shapes(vertices_polygons, im.shape[2:])

        # calculate correction for mask
        im_divided = fill_gaps_between_cells(mask_shapes_overlap)
        mask_corrected = mask_shapes + im_divided

        # calculate corrected polygons
        shapes_corrected = []

        for level in range(np.max(mask_corrected)):

            contour = find_contours(mask_corrected==level+1,0.5)[0]

            coords = approximate_polygon(contour, tolerance=1)
        
            shapes_corrected.append(coords)

    else:

        shapes_corrected = vertices_polygons
    
    # visualize corrected polygons
    if ('Shapes corrected' in [x.name for x in viewer.layers]):

        viewer.layers['Shapes corrected'].data = shapes_corrected

    else:
        viewer.add_shapes(shapes_corrected, shape_type='polygon',name='Shapes corrected')

    viewer.status = 'Polygons corrected.'

In [5]:
def save_shapes_and_mask(viewer: Viewer):

    global path_save
    global im_name
    global im
    
    # get coordinates of vertices from corrected layer if possible
    if ('Shapes corrected' in [x.name for x in viewer.layers]):

        vertices_polygons = viewer.layers['Shapes corrected'].data

    else:

        vertices_polygons = viewer.layers['Shapes'].data

    
    # save vertices
    pkl_path = os.path.join(path_save,im_name.replace('.nd2','_polygons.pkl'))
    with open(pkl_path, 'wb') as f:
        pickle.dump(vertices_polygons, f)

    
    # create mask from vertices
    _, mask = create_mask_from_shapes(vertices_polygons, im.shape[2:])

    # display mask
    viewer.add_labels(mask)
    
    # save mask
    mask_path = os.path.join(path_save,im_name.replace('.nd2','_mask.png'))
    imsave(mask_path,mask)

    
    viewer.status = 'Data has been saved.'

In [4]:
path_dir = r'D:\data_analysis\2022_Sahana\data\Collagen\60x images'

path_save = r'D:\data_analysis\2022_Sahana\masks'

In [5]:
# find a list of available files
# and print its length
list_files = os.listdir(path_dir)
len(list_files)

16

In [6]:
# ideas to change the length of the list

# choose n first elements
list_files = list_files[:2]
len(list_files)

2

In [7]:
# loop over all files found in the loop

for im_name in list_files:

    # create a full pathway 
    path_im = os.path.join(path_dir,im_name)

    # open image
    im = nd2.imread(path_im)

    # create a maximum projection
    actin_max = np.max(im[:,0,:,:],axis=0)

    # create a viewer to draw shapes
    viewer = napari.Viewer()
    
    viewer.add_image(im[:,0,:,:],colormap='magenta',blending='additive')
    viewer.add_image(im[:,1,:,:],colormap='red',blending='additive')
    viewer.add_image(im[:,2,:,:],colormap='green',blending='additive')
    viewer.add_image(im[:,3,:,:],colormap='blue',blending='additive')
    viewer.add_image(actin_max,blending='additive')
    
    # open existing shapes or add an empty layer
    pkl_path = os.path.join(path_save,im_name.replace('.nd2','_polygons.pkl'))

    if os.path.exists(pkl_path):

        # open existing mask
        with open(pkl_path, 'rb') as f:
            vertices_polygons = pickle.load(f)

        viewer.add_shapes(vertices_polygons, shape_type='polygon',name='Shapes')

    else:
        viewer.add_shapes()


    # add polygons correction button
    correct_data = magicgui(correct_shapes, call_button='Refine shapes')
    viewer.window.add_dock_widget(correct_data,area='left')

    # add saving button to napari
    save_data = magicgui(save_shapes_and_mask, call_button='Save Data')
    viewer.window.add_dock_widget(save_data,area='left')

    # we will wait for the user to draw shapes here
    viewer.show(block=True)


Assistant skips harvesting pyclesperanto as it's not installed.


c:\Users\lab\.conda\envs\napari-env\lib\site-packages\numpy\core\numeric.py:2463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
c:\Users\lab\.conda\envs\napari-env\lib\site-packages\numpy\core\numeric.py:2463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
